Rough implementation of the classic arXiv paper, word2vec.

In [13]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sn

In [14]:
text = "the cat sat on the mat"

In [15]:
for i in text:
    print(i)
    break

t


In [16]:
!pip install nltk



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [17]:
import nltk
nltk.download('brown')


[nltk_data] Downloading package brown to
[nltk_data]     /Users/kaushikdwivedi/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [18]:
from nltk.corpus import brown


In [19]:
brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [20]:
text = "the cat sat on the mat"

In [21]:
brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [22]:
for l in brown.words():
    print(l)
    break

The


In [29]:
len(brown.words())

1161192

In [30]:
words_small = list(brown.words())[:100000]

In [34]:
len(words_small)

100000

In [35]:
#we're starting our work with a 100,000 words for easier training
words = words_small

In [36]:
#building the vocabulary
#we first need to tokenise: remove punctuations, and turn capital letters into small


In [37]:
sample_text = ["The", "Cat's", "Dad", "sat", "on", "the", "mat"]

In [44]:
for word in sample_text:
    print(word.lower())

the
cat's
dad
sat
on
the
mat


In [45]:
import re

In [48]:
tokenizer_rndn = []
for word in sample_text:
    clean_parts = re.findall(r'\w+', word)
    tokenizer_rndn.extend(clean_parts)

In [49]:
tokenizer_rndn

['The', 'Cat', 's', 'Dad', 'sat', 'on', 'the', 'mat']

In [50]:
for word in tokenizer_rndn:
    word.lower()
    
    

In [51]:
tokenizer_rndn

['The', 'Cat', 's', 'Dad', 'sat', 'on', 'the', 'mat']

In [295]:
#words

In [53]:
import re

In [54]:
def tokenize(word_list):
    tokenized_words = []
    for word in word_list:
        
        word = word.lower()
        word = re.sub(r"'", "", word)
        
        clean_parts = re.findall(r"\w+", word)
        tokenized_words.extend(clean_parts)
    
    return tokenized_words

In [64]:
words = tokenize(words)

In [65]:
" ' " in words

False

In [67]:
"!" in words

False

In [68]:
"'" in words

False

In [69]:
"dont" in words

True

In [70]:
#done w the tokenizer, time to build the vocab
vocab = {}

In [72]:
len(words)

89563

In [78]:
j =0 


In [79]:
j

0

In [80]:
while(j<len(words)):
    vocab[j] = words[j]
    j=j+1

In [82]:
len(vocab)

89563

In [83]:
#since our dict is ready, we're now gonna create our emb table

In [84]:
emb_dim = 100
vocab_size = len(vocab)

In [85]:
emb_table = np.random.rand(vocab_size, emb_dim)

In [296]:
#emb_table

In [90]:
 #rough
'the' in vocab

False

In [92]:
0 in vocab

True

In [93]:
index_to_word = vocab

In [94]:
word_to_index = {word: idx for idx, word in vocab.items()}

In [297]:
#word_to_index

In [96]:
'the' in word_to_index

True

In [97]:
sentence = ["the", "cat", "is", "brown"]

In [98]:
for i in sentence:
    print(word_to_index[i])

89559


KeyError: 'cat'

In [99]:
#seems like our dataset doesnt have the word cat
#trying some other sentence
#trying to mimic some words

sentence = ["the", "election", "was", "grand"]

In [100]:
for i in sentence:
    print(word_to_index[i])

89559
75708
89024
80451


In [101]:
word_to_index['the']

89559

In [102]:
index_to_word[89559]

'the'

In [111]:
#rough
tokenized_corpus = []
for i in index_to_word:
    tokenized_corpus.append(i)

    

In [113]:
len(tokenized_corpus)

89563

In [236]:
class Pair:
    
    def __init__(self, center_idx, context_idx):
        self.center_idx = center_idx
        self.context_idx = context_idx
        
    def get_center(self):
        return self.center_idx
        
    def get_context(self):
        return self.context_idx
    
    def as_list(self):
        return [self.center_dx]
    
    def __str__(self):
        return f"({self.center_idx}, {self.context_idx})"

    def __repr__(self):
        return self.__str__()
        
    
        
        

In [237]:
p1 = Pair(5,6)

In [238]:
p1.get_center()

5

In [239]:
p2 = Pair(10,11)

In [240]:
for i in range(5):
    Pair(i, i+1)

In [241]:
len(tokenized_corpus)

89563

In [243]:
list_pairs =[]


In [244]:
p1 = Pair(4,6)
p2 = Pair(4,5)
p3 = Pair(4,3)
p4 = Pair(4,2)

In [248]:
for pair in pairs:
    print(pair)

(4, 6)
(4, 5)
(4, 3)
(4, 2)


In [249]:
class PairDataset:
    
    def __init__(self, pair_list):
        self.pairs = pair_list
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self,idx):
        return self.pairs[idx]
    
    def __str__(self):
        return f"PairDataset of {len(self)} pairs)"

In [250]:
s_w = 2

In [251]:
s_w = 2  # Define your window size (assumed 2 based on context)
pair_list = []

for i in range(len(tokenized_corpus)):
    # Handle the first word
    if i == 0:
        pair_list.append(Pair(i, i+1))
        pair_list.append(Pair(i, i+2))
        
    # Handle the second word
    elif i == 1:
        pair_list.append(Pair(i, i-1))
        pair_list.append(Pair(i, i+1))
        pair_list.append(Pair(i, i+2))
    
    # Handle the last word
    elif i == len(tokenized_corpus) - 1:
        pair_list.append(Pair(i, i-1))
        pair_list.append(Pair(i, i-2))
    
    # Handle the second-to-last word
    elif i == len(tokenized_corpus) - 2:
        pair_list.append(Pair(i, i+1))
        pair_list.append(Pair(i, i-1))
        pair_list.append(Pair(i, i-2))
        
    # Handle all other words (with a sliding window)
    else:
        # Words to the right
        if i + s_w < len(tokenized_corpus):
            pair_list.append(Pair(i, i+s_w))
        if i + s_w - 1 < len(tokenized_corpus):
            pair_list.append(Pair(i, i+s_w-1))
        
        # Words to the left
        if i - s_w >= 0:
            pair_list.append(Pair(i, i-s_w))
        if i - s_w + 1 >= 0:
            pair_list.append(Pair(i, i-s_w+1))

In [271]:
dataset = PairDataset(pair_list)

In [253]:
dataset[5]

(2, 4)

In [286]:
for pair in dataset:
    center_idx = pair.center_idx
    context_idx = pair.context_idx
    
    print(center_idx)
    break

0


In [283]:
dataset[3]

(1, 2)

In [269]:
dataset[1]

(0, 2)

In [254]:
dataset[0]

(0, 1)

In [259]:
w=np.random.rand(vocab_size,emb_dim)

In [261]:
import random

In [265]:
def multiply(a,b):
    c = a*b
    return a,b

In [266]:
multiply(3,3)

(3, 3)

In [287]:
arr1 = np.random.rand(3,3)
arr2 = np.random.rand(3,3)

In [288]:
eg = np.dot(arr1[0], arr2[0])
eg

np.float64(0.8591674644194397)

In [264]:
import numpy as np

def sigmoid(x):
    
    # Clip values to avoid overflow in exp
    x_clipped = np.clip(x, -20, 20)
    
    # Compute sigmoid
    return 1 / (1 + np.exp(-x_clipped))

In [290]:
list =[]
for i in range(100):
    list.append(i)

In [292]:
import random

In [293]:
j = random.choice(list)
j

55

In [294]:
class Word2Vec:
    
    def __init__(self, vocab_size, emb_dim, dataset, learning_rate=0.01, neg_samples = 5):
        
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.learning_rate = learning_rate
        self.neg_samples = neg_samples
        self.dataset = dataset
        
        self.W_in = np.random.rand(vocab_size, emb_dim)
        self.W_out = np.random.rand(vocab_size, emb_dim)
    
        
    def forward(self):
        
        positive_prob = []
        negative_prob = []
        
        for pair in self.dataset:
            center_idx = pair.center_idx
            context_idx = pair.context_idx
            
            #positive prob
            pos_prob = np.dot(self.W_in[center_idx], self.W_out[context_idx])
            pos_prob = sigmoid(pos_prob)
            positive_prob.append(pos_prob)

            #negative prob
            all_indices = list(range(self.vocab_size))
            all_indices.remove(pair.context_idx)
            
            neg_probs_per_pair = []
            
            for i in range(self.neg_samples):
                neg_idx = random.choice(all_indices)
                neg_score = np.dot(self.W_in[center_idx], self.W_out[neg_idx])
                neg_prob = sigmoid(-neg_score)
                neg_probs_per_pair.append(neg_prob)
                
            negative_prob.append(neg_probs_per_pair)
                
        positive_prob_array = np.array(positive_prob)
        negative_prob_array = np.array(negative_prob)
        
        pos_loss_total = -np.log(positive_prob_array)
        neg_loss_total = -np.log(negative_prob_array)
        
        total_loss_per_pair = pos_loss_total + neg_loss_total
        
        #mean loss:
        loss = np.mean(total_loss_per_pair)        
        
        